In [ ]:
import os
import streamlit as st
import pickle
import time
from PyPDF2 import PdfReader
import google.generativeai as genai # For Using Google Gemini pro model
from langchain.document_loaders import UnstructuredPDFLoader #Load the PDF from langchain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
from google.colab import files #Just the upload files in google colab
from langchain.document_loaders import PyPDFLoader #Load the PDF from langchain
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
genai.configure(api_key= "....")

In [ ]:
# Load the PDF using Langchain's PyPDFLoader
loader = PyPDFLoader("./pdf_path")
documents = loader.load() #Thid documents will be given to the recursive Document splitter

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ",", "}", "{", ":", " "],
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len,
)
texts = splitter.split_documents(documents)

In [ ]:
embeddings_google = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstoregoogleai = FAISS.from_documents(texts, embeddings_google) #Performing openai embeding. We can also go google embedings
vectorstoregoogleai.save_local("faiss_index_googleai")

In [ ]:
question = '''
Any infirmation you wanna exreact from the document?
'''

vectorstoregoogleai = FAISS.load_local("faiss_index_openai", embeddings_google, allow_dangerous_deserialization=True)
answerfromvectordb = vectorstoregoogleai.similarity_search(question) #Storing the similar kind of vectors according to the question from the main vector database

In [ ]:
llm = ChatGoogleGenerativeAI(model = "gemini-pro")

In [ ]:
chain = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, retriever=vectorstoregoogleai.as_retriever()) #Chreating a chanin of vector data where we can ask question

In [ ]:
response = chain({"question": question}, answerfromvectordb) #Retriving anser from the vector database

In [ ]:
response["answer"]